In [1]:

# !pip install keras==2.3.1
# !pip install git+https://www.github.com/keras-team/keras-contrib.git
# !pip install patchify    #To install and import other mentioned libraries  in code
# !pip install segmentation_models
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, Conv1D
from tensorflow.keras.layers import MaxPooling2D, MaxPooling1D, Reshape
from tensorflow.keras import backend as k
from keras.layers import BatchNormalization
import torch
from sklearn.metrics import classification_report

import optuna
from keras.models import Sequential
from keras.layers import Reshape, Conv1D, MaxPooling1D, BatchNormalization, Dense, Dropout, Bidirectional, LSTM
import tensorflow as tf
from optuna.integration import TFKerasPruningCallback
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

2023-12-03 16:28:46.962513: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 16:28:46.962737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 16:28:46.987371: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 16:28:47.039844: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-03 16:28:47.913610: W tensorflow/compiler/tf2

In [2]:
from tensorflow.compat.v1.keras.layers import CuDNNLSTM


In [3]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dense, Activation, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from transformers import BertTokenizer, TFBertModel
import numpy as np
import os
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [4]:
os.environ["WANDB_API_KEY"] = "0" ## to silence warning

In [5]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


## Formatando o Dataset

In [6]:
max_lenght = 100
X = np.load(f'../../dataset/processed/artigos_tratados/bertimbau/bertimbau_full__max_lenght={max_lenght}.npy')
data_raw = pd.read_parquet('../../dataset/processed/artigos_tratados/artigos_tratados.parquet')

In [7]:
indexes = pd.read_csv('../../dataset/index_splits.csv', index_col = 0)

In [8]:
indexes

,name,value
0,linhas_sem_texto,"[1140, 1165, 1188, 1314, 1320, 1339, 1361, 200..."
1,strat_vies,"[[2596, 1763, 6620, 10805, 3029, 2637, 744, 41..."
2,strat_part,"[[4465, 2218, 972, 580, 1957, 5448, 6819, 8621..."
3,part_novos,"[[5497, 6570, 4197, 1214, 5300, 2491, 5284, 38..."


In [9]:
y = data_raw['Vies']

y = y.map({
    'direita':2,
    'centro':1,
    'esquerda': 0})

y = pd.get_dummies(y).values

## Criando Classificador

In [10]:
unique, counts = np.unique(y.argmax(axis=1), return_counts=True)
esquerda_count, centro_count, direita_count  = counts
total_count = sum(counts)

In [11]:
num_classes = 3
class_weight = {0: (1 / esquerda_count) * (total_count / num_classes),
                1: (1 / centro_count) * (total_count / num_classes),
                2: (1 / direita_count) * (total_count / num_classes)}

In [12]:
METRICS = [
      # keras.metrics.TruePositives(name='tp'),
      # keras.metrics.FalsePositives(name='fp'),
      # keras.metrics.TrueNegatives(name='tn'),
      # keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      ]

2023-12-03 16:28:56.424161: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-03 16:28:56.428015: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-03 16:28:56.428196: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

### Teste inicial 

In [13]:
def build_lstm_model(input_shape, output_shape, metrics):
    tf.keras.backend.clear_session()

    model = tf.keras.Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=3),
        tf.keras.layers.Dropout(0.5),
        Bidirectional(CuDNNLSTM(100, kernel_regularizer=tf.keras.regularizers.l2(0.01), return_sequences=True)),
        tf.keras.layers.Dropout(0.5),
        Bidirectional(CuDNNLSTM(50, kernel_regularizer=tf.keras.regularizers.l2(0.01), return_sequences=True)),
        tf.keras.layers.Dropout(0.5),
        MaxPooling1D(pool_size=3),
        Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(output_shape, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(2e-5),
                metrics=METRICS)
    
    return model

#### Strat Vies 

In [15]:
index_strat_vies = eval(indexes.loc[1,'value'])
train_strat_vies_index, val_strat_vies_index, test_strat_vies_index = index_strat_vies

In [16]:
X_train_strat_vies = X[train_strat_vies_index]
y_train_strat_vies = y[train_strat_vies_index]

X_val_strat_vies = X[val_strat_vies_index]
y_val_strat_vies = y[val_strat_vies_index]

X_test_strat_vies = X[test_strat_vies_index]
y_test_strat_vies = y[test_strat_vies_index]

In [18]:
model = build_lstm_model(
    input_shape = (X_train_strat_vies.shape[1], X_train_strat_vies.shape[2]),
    output_shape = 3,
    metrics = METRICS
    )

history = model.fit(x=X_train_strat_vies, y=y_train_strat_vies, 
                    epochs=100,
                    batch_size=8,
                    class_weight=class_weight,
                    validation_data=(X_val_strat_vies, y_val_strat_vies),
                    callbacks=[
                        EarlyStopping(monitor='val_prc', patience=10, mode='max', restore_best_weights=True)],
                    verbose=True)

2023-12-03 15:14:05.176653: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2446233600 exceeds 10% of free system memory.
2023-12-03 15:14:24.098228: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2446233600 exceeds 10% of free system memory.


Epoch 1/100


2023-12-03 15:14:28.350195: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2023-12-03 15:14:28.622120: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.82GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:14:28.646892: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.82GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:14:30.059630: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f7aa008a9d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-03 15:14:30.059649: I external/local_xla/xla/service/service.

  6/996 [..............................] - ETA: 10s - loss: 6.2129 - precision: 0.6667 - recall: 0.1250 - auc: 0.6089 - prc: 0.5021    

2023-12-03 15:14:31.617309: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:14:31.617354: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:14:31.630261: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:14:31.630300: W external/local_tsl/tsl/framework/bfc_

996/996 [==============================] - ETA: 0s - loss: 5.3770 - precision: 0.7931 - recall: 0.3313 - auc: 0.7960 - prc: 0.6868

2023-12-03 15:14:41.641796: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.35GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:14:41.657818: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.35GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:14:41.675852: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:14:41.683432: W external/local_tsl/tsl/framework/bfc_

996/996 [==============================] - 20s 14ms/step - loss: 5.3770 - precision: 0.7931 - recall: 0.3313 - auc: 0.7960 - prc: 0.6868 - val_loss: 4.4247 - val_precision: 0.8438 - val_recall: 0.7627 - val_auc: 0.9328 - val_prc: 0.8946
Epoch 2/100
996/996 [==============================] - 10s 10ms/step - loss: 3.9000 - precision: 0.8346 - recall: 0.7752 - auc: 0.9308 - prc: 0.8831 - val_loss: 3.3498 - val_precision: 0.8631 - val_recall: 0.8124 - val_auc: 0.9526 - val_prc: 0.9205
Epoch 3/100
996/996 [==============================] - 10s 10ms/step - loss: 3.0195 - precision: 0.8561 - recall: 0.8194 - auc: 0.9482 - prc: 0.9122 - val_loss: 2.5956 - val_precision: 0.8756 - val_recall: 0.8429 - val_auc: 0.9652 - val_prc: 0.9403
Epoch 4/100
996/996 [==============================] - 10s 10ms/step - loss: 2.3971 - precision: 0.8640 - recall: 0.8331 - auc: 0.9552 - prc: 0.9246 - val_loss: 2.0927 - val_precision: 0.8817 - val_recall: 0.8588 - val_auc: 0.9656 - val_prc: 0.9416
Epoch 5/100
996/

In [22]:
y_pred_proba = model.predict(X_test_strat_vies, batch_size=8)
y_pred = y_pred_proba.argmax(axis =1)
print(classification_report(y_test_strat_vies.argmax(axis=1), y_pred))

              precision    recall  f1-score   support

           0       0.88      0.86      0.87       474
           1       0.89      0.91      0.90       704
           2       0.97      0.97      0.97      1035

    accuracy                           0.93      2213
   macro avg       0.91      0.91      0.91      2213
weighted avg       0.93      0.93      0.93      2213



#### Strat part 

In [14]:
index_strat_part = eval(indexes.loc[2,'value'])
train_strat_part_index, val_strat_part_index,test_strat_part_index  = index_strat_part

X_train_strat_part = X[train_strat_part_index]
y_train_strat_part = y[train_strat_part_index]

X_val_strat_part = X[val_strat_part_index]
y_val_strat_part = y[val_strat_part_index]

X_test_strat_part = X[test_strat_part_index]
y_test_strat_part = y[test_strat_part_index]

In [15]:
model = build_lstm_model(
    input_shape = (X_train_strat_part.shape[1], X_train_strat_part.shape[2]),
    output_shape = 3,
    metrics = METRICS
    )

history = model.fit(x=X_train_strat_part, y=y_train_strat_part, 
                    epochs=100,
                    batch_size=8,
                    class_weight=class_weight,
                    validation_data=(X_val_strat_part, y_val_strat_part),
                    callbacks=[
                        EarlyStopping(monitor='val_prc', patience=10, mode='max', restore_best_weights=True)],
                    verbose=True)

2023-12-03 15:44:50.199242: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2446233600 exceeds 10% of free system memory.
2023-12-03 15:45:14.086699: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2446233600 exceeds 10% of free system memory.


Epoch 1/100


2023-12-03 15:45:18.574866: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2023-12-03 15:45:18.852233: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.82GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:45:18.878061: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.82GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:45:20.317746: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fbf486f30d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-03 15:45:20.317764: I external/local_xla/xla/service/service.

  6/996 [..............................] - ETA: 10s - loss: 6.4181 - precision: 0.4545 - recall: 0.1042 - auc: 0.4890 - prc: 0.3515            

2023-12-03 15:45:21.877117: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:45:21.877170: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:45:21.890876: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:45:21.890937: W external/local_tsl/tsl/framework/bfc_

996/996 [==============================] - ETA: 0s - loss: 5.3868 - precision: 0.7638 - recall: 0.4211 - auc: 0.8135 - prc: 0.7071

2023-12-03 15:45:31.886633: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.35GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:45:31.902570: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.35GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:45:31.920645: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 15:45:31.928680: W external/local_tsl/tsl/framework/bfc_

996/996 [==============================] - 20s 14ms/step - loss: 5.3868 - precision: 0.7638 - recall: 0.4211 - auc: 0.8135 - prc: 0.7071 - val_loss: 4.4785 - val_precision: 0.8570 - val_recall: 0.7582 - val_auc: 0.9352 - val_prc: 0.8952
Epoch 2/100
996/996 [==============================] - 10s 11ms/step - loss: 4.0005 - precision: 0.8447 - recall: 0.7844 - auc: 0.9332 - prc: 0.8888 - val_loss: 3.4701 - val_precision: 0.8528 - val_recall: 0.8181 - val_auc: 0.9493 - val_prc: 0.9153
Epoch 3/100
996/996 [==============================] - 11s 11ms/step - loss: 3.1350 - precision: 0.8617 - recall: 0.8246 - auc: 0.9481 - prc: 0.9132 - val_loss: 2.7476 - val_precision: 0.8634 - val_recall: 0.8215 - val_auc: 0.9574 - val_prc: 0.9287
Epoch 4/100
996/996 [==============================] - 11s 11ms/step - loss: 2.4961 - precision: 0.8711 - recall: 0.8381 - auc: 0.9564 - prc: 0.9259 - val_loss: 2.1945 - val_precision: 0.8672 - val_recall: 0.8339 - val_auc: 0.9633 - val_prc: 0.9375
Epoch 5/100
996/

In [16]:
y_pred_proba = model.predict(X_test_strat_part, batch_size=8)
y_pred = y_pred_proba.argmax(axis =1)
print(classification_report(y_test_strat_part.argmax(axis=1), y_pred))

2023-12-03 15:53:18.285178: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 679833600 exceeds 10% of free system memory.


277/277 [==============================] - 2s 4ms/step
              precision    recall  f1-score   support

           0       0.87      0.86      0.86       474
           1       0.88      0.90      0.89       705
           2       0.97      0.96      0.97      1034

    accuracy                           0.92      2213
   macro avg       0.91      0.91      0.91      2213
weighted avg       0.92      0.92      0.92      2213



#### Part novos 

In [14]:
index_part_novos = eval(indexes.loc[3,'value'])
train_part_novos_index, val_part_novos_index, test_part_novos_index = index_part_novos

X_train_part_novos = X[train_part_novos_index]
y_train_part_novos = y[train_part_novos_index]

X_val_part_novos = X[val_part_novos_index]
y_val_part_novos = y[val_part_novos_index]

X_test_part_novos = X[test_part_novos_index]
y_test_part_novos = y[test_part_novos_index]

In [15]:
pd.DataFrame(y_test_part_novos.argmax(axis = 1)).value_counts()

2    1161
1     814
0     622
Name: count, dtype: int64

In [16]:
model = build_lstm_model(
    input_shape = (X_train_part_novos.shape[1], X_train_part_novos.shape[2]),
    output_shape = 3,
    metrics = METRICS
    )

history = model.fit(x=X_train_part_novos, y=y_train_part_novos, 
                    epochs=100,
                    batch_size=8,
                    class_weight=class_weight,
                    validation_data=(X_val_part_novos, y_val_part_novos),
                    callbacks=[
                        EarlyStopping(monitor='val_prc', patience=10, mode='max', restore_best_weights=True)],
                    verbose=True)

2023-12-03 16:31:45.203039: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2339942400 exceeds 10% of free system memory.
2023-12-03 16:32:03.023255: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2339942400 exceeds 10% of free system memory.


Epoch 1/100


2023-12-03 16:32:07.200349: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2023-12-03 16:32:07.482764: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.82GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 16:32:07.508374: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.82GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 16:32:08.921606: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f27a815b820 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-03 16:32:08.921660: I external/local_xla/xla/service/service.

  6/953 [..............................] - ETA: 10s - loss: 6.3140 - precision: 0.6667 - recall: 0.0833 - auc: 0.5664 - prc: 0.4170            

2023-12-03 16:32:10.463034: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 16:32:10.463081: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 16:32:10.476504: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 16:32:10.476548: W external/local_tsl/tsl/framework/bfc_

953/953 [==============================] - ETA: 0s - loss: 5.4411 - precision: 0.7611 - recall: 0.3379 - auc: 0.7941 - prc: 0.6815

2023-12-03 16:32:19.964141: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.16GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 16:32:19.976894: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.16GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 16:32:20.006733: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-12-03 16:32:20.014635: W external/local_tsl/tsl/framework/bfc_

953/953 [==============================] - 19s 14ms/step - loss: 5.4411 - precision: 0.7611 - recall: 0.3379 - auc: 0.7941 - prc: 0.6815 - val_loss: 4.5265 - val_precision: 0.8934 - val_recall: 0.7521 - val_auc: 0.9481 - val_prc: 0.9181
Epoch 2/100
953/953 [==============================] - 10s 10ms/step - loss: 4.0489 - precision: 0.8220 - recall: 0.7491 - auc: 0.9282 - prc: 0.8798 - val_loss: 3.4740 - val_precision: 0.8702 - val_recall: 0.8312 - val_auc: 0.9595 - val_prc: 0.9347
Epoch 3/100
953/953 [==============================] - 10s 10ms/step - loss: 3.1645 - precision: 0.8499 - recall: 0.8153 - auc: 0.9483 - prc: 0.9119 - val_loss: 2.7581 - val_precision: 0.8735 - val_recall: 0.8394 - val_auc: 0.9643 - val_prc: 0.9413
Epoch 4/100
953/953 [==============================] - 10s 10ms/step - loss: 2.5301 - precision: 0.8651 - recall: 0.8368 - auc: 0.9570 - prc: 0.9272 - val_loss: 2.2133 - val_precision: 0.8954 - val_recall: 0.8595 - val_auc: 0.9715 - val_prc: 0.9517
Epoch 5/100
953/

In [17]:
y_pred_proba = model.predict(X_test_part_novos, batch_size=8)
y_pred = y_pred_proba.argmax(axis =1)
print(classification_report(y_test_part_novos.argmax(axis=1), y_pred))

2023-12-03 16:42:39.315374: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 797798400 exceeds 10% of free system memory.


325/325 [==============================] - 2s 4ms/step
              precision    recall  f1-score   support

           0       0.51      0.60      0.55       622
           1       0.63      0.54      0.58       814
           2       0.92      0.93      0.93      1161

    accuracy                           0.73      2597
   macro avg       0.69      0.69      0.69      2597
weighted avg       0.73      0.73      0.73      2597

